In [19]:
import pandas as pd
from herbie import Herbie, FastHerbie


date_time: str = '2021-01-01 1:31:19.2913102301293019230'
model = 'hrrr'
product = 'sfc'  # surface fields

H = Herbie(
    date_time,        # date & time
    model=model,      # weather prediction model
    product=product,
    fxx=6             # forecast lead time: 6 hours
)

# Each hour for the last 6 hours
# DATES = pd.date_range(
#     start=date_time,
#     periods=6,
#     freq="1h"
# )

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Jan-01 01:31 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws


In [20]:
temp = H.xarray(":(?:TMP):")


Note: Returning a list of [3] xarray.Datasets because cfgrib opened with multiple hypercubes.


In [21]:
start_date, start_time = str(temp[0].time.data).split("T")
start_time

'01:00:00.000000000'

In [14]:
start_date, start_time = str(temp[0].time.data).split("T")
start_time

'12:00:00.000000000'

In [27]:
import cdsapi
c = cdsapi.Client()
c.retrieve('reanalysis-era5-single-levels',
           {
               'product_type': 'reanalysis',
               'format': 'grib',
           },
           'download.grib')

Exception: Missing/incomplete configuration file: C:\Users\camer/.cdsapirc

In [44]:
import cdsapi

cds = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2", key = "308352:704f911a-0b2a-427f-ac5b-c9afd4084f09")
cds.retrieve('reanalysis-era5-pressure-levels', {
           "variable": "temperature",
           "pressure_level": "1000",
           "product_type": "reanalysis",
           "date": "2017-12-01/2017-12-31",
           "time": "12:00",
           "format": "netcdf"
       }, 'download.nc')

2024-05-06 20:24:00,925 DEBUG CDSAPI {'url': 'https://cds.climate.copernicus.eu/api/v2', 'key': '308352:704f911a-0b2a-427f-ac5b-c9afd4084f09', 'quiet': False, 'verify': True, 'timeout': 60, 'progress': True, 'sleep_max': 120, 'retry_max': 500, 'full_stack': False, 'delete': True, 'metadata': None, 'forget': False}
2024-05-06 20:24:01,959 INFO Welcome to the CDS
2024-05-06 20:24:01,959 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-05-06 20:24:01,960 DEBUG POST https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels {"variable": "temperature", "pressure_level": "1000", "product_type": "reanalysis", "date": "2017-12-01/2017-12-31", "time": "12:00", "format": "netcdf"}
2024-05-06 20:24:02,205 DEBUG REPLY {'state': 'queued', 'request_id': 'efda1347-95c1-4b88-86a1-935c4c838da3', 'specific_metadata_json': {'top_request_origin': 'api'}}
2024-05-06 20:24:02,206 INFO Request is queued
2024-05-06 20:24:02

Result(content_length=64380752,content_type=application/x-netcdf,location=https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data4/adaptor.mars.internal-1715052244.4015183-1297-19-efda1347-95c1-4b88-86a1-935c4c838da3.nc)

In [48]:
import pandas as pd
df = pd.read_csv(r'C:\Users\camer\Downloads\WeatherV4.csv', sep='@')
date_list = df['SampledDate'].tolist()
date_list = '/'.join(date_list)
print(date_list[:50])
cds.retrieve('reanalysis-era5-pressure-levels', {
           "variable": "temperature",
           "pressure_level": "1000",
           "product_type": "reanalysis",
           "date": date_list,
           "time": "12:00",
           "format": "netcdf"
       }, 'download.nc')

2020-11-26/2018-06-05/1998-06-28/2001-08-23/2014-1


In [52]:
import netCDF4
import os
cwd = os.getcwd()
weather_file = netCDF4.Dataset(cwd+'\download.nc','r')
print(weather_file.variables.keys())
print(weather_file)

dict_keys(['longitude', 'latitude', 'time', 't'])
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2024-05-07 03:24:05 GMT by grib_to_netcdf-2.28.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf -S param -o /cache/data4/adaptor.mars.internal-1715052244.4015183-1297-19-efda1347-95c1-4b88-86a1-935c4c838da3.nc /cache/tmp/efda1347-95c1-4b88-86a1-935c4c838da3-adaptor.mars.internal-1715052242.1799798-1297-15-tmp.grib
    dimensions(sizes): longitude(1440), latitude(721), time(31)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 t(time, latitude, longitude)
    groups: 


In [56]:
time = weather_file.variables['time']
temp = weather_file.variables['t']
print("time:")
print(time)
print("\n")
print("temp")
print(temp)

time:
<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (31,)
filling on, default _FillValue of -2147483647 used


temp
<class 'netCDF4._netCDF4.Variable'>
int16 t(time, latitude, longitude)
    scale_factor: 0.0014835409065733103
    add_offset: 268.79999370317955
    _FillValue: -32767
    missing_value: -32767
    units: K
    long_name: Temperature
    standard_name: air_temperature
unlimited dimensions: 
current shape = (31, 721, 1440)
filling on
